In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# miniFraudFinder - Exploratory Data Analysis

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?download_url=https://github.com/jrdetorre-google/minifraudfinder/raw/main/01_exploratory_data_analysis.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/jrdetorre-google/minifraudfinder/blob/main/01_exploratory_data_analysis.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/jrdetorre-google/minifraudfinder/blob/main/01_exploratory_data_analysis.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[miniFraudFinder](https://github.com/jrdetorre-google/minifraudfinder) is a series of labs on how to build a  fraud detection system on Google Cloud. Throughout the miniFraudFinder labs, you will learn how to read historical bank transaction data stored in data warehouse,  perform exploratory data analysis (EDA), do feature engineering,  train a model u, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model and monitor your model.

### Objective

In this notebook, you will perform exploratory data analysis on the historical bank transactions stored in BigQuery. Please make sure you have completed the [environment setup notebook](00_environment_setup.ipynb) prior to running this notebook.

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)
- [Google Cloud Storage](https://cloud.google.com/storage)

Steps performed in this notebook:

- Read data from BigQuery
- Calculate summary statistics across historical transactions
- Calculate fraud percentages
- Plot distributions of transaction amounts
- Analyze customer-level aggregates of transaction data

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Setup based on notebook environment

If using Colab, make sure to enter your project ID in the cell below.

In [3]:
# Detect if current environment is Colab, authenticate if so
# Enter in project directly if Colab, detect from environment otherwise
# Also adjusts plotly renderer for Colab vs other (to display interactive plots)

import sys
import plotly.io as pio

if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

    # If Colab, enter in Cloud project here
    GCP_PROJECTS = ['YOUR-PROJECT-ID'] # @param {type:"raw"}

    pio.renderers.default = 'colab'

else:
    # Detect Cloud project from environment
    GCP_PROJECTS = !gcloud config get-value project

    pio.renderers.default = 'iframe'

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [4]:
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)


BUCKET_NAME          = "orange-ml-demo-fraudfinder"
PROJECT              = "orange-ml-demo"
REGION               = "us-central1"
ID                   = "zv7r3"
FEATURESTORE_ID      = "fraudfinder_zv7r3"
MODEL_NAME           = "ff_model"
ENDPOINT_NAME        = "ff_model_endpoint"
TRAINING_DS_SIZE     = "1000"



### Import libraries

In [10]:
from typing import Union

import pandas as pd
import bigframes.pandas as bpd
import plotly.express as px
from google.cloud import bigquery

#### Initialize BigQuery SDK for Python 

Use a helper function for sending queries to BigQuery.

In [6]:
# Wrapper to use BigQuery client to run query/job, return job ID or result as DF
def run_bq_query(sql: str) -> Union[str, pd.DataFrame]:
    """
    Run a BigQuery query and return the job ID or result as a DataFrame
    Args:
        sql: SQL query, as a string, to execute in BigQuery
    Returns:
        df: DataFrame of results from query,  or error, if any
    """

    bq_client = bigquery.Client(project = PROJECT_ID)

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

### Exploratory data analysis of transaction data in BigQuery

In this section, you'll explore some of the FraudFinder data by running queries and creating a couple interactive plots.

#### Get transaction data summary statistics
First, you generate some summary statistics across the different fields in the transaction data.

In [7]:
run_bq_query(
    """
    SELECT
      COUNT(*) AS NUM_TX,

      MIN(TX_TS) AS MIN_TX_DATE,
      MAX(TX_TS) AS MAX_TX_DATE,

      COUNT(DISTINCT CUSTOMER_ID) AS NUM_CUSTOMERS,
      COUNT(DISTINCT TERMINAL_ID) AS NUM_TERMINALS,

      MIN(TX_AMOUNT) AS MIN_TX_AMOUNT,
      AVG(TX_AMOUNT) AS AVG_TX_AMOUNT,
      MAX(TX_AMOUNT) AS MAX_TX_AMOUNT

    FROM
      tx.tx
    """
)

Finished job_id: a82f46be-33f6-499d-a735-8195b0ec9f92


,NUM_TX,MIN_TX_DATE,MAX_TX_DATE,NUM_CUSTOMERS,NUM_TERMINALS,MIN_TX_AMOUNT,AVG_TX_AMOUNT,MAX_TX_AMOUNT
0,73968721,2022-01-01 00:00:00+00:00,2024-01-01 00:12:40+00:00,49984,5000,0E-9,55.152796713,1101.550000000


The results show that there are over 148M transactions spanning the course of 2022 and 2025, taking place at 5000 different terminals with nearly 50K unique customers. The transaction amounts vary from 0 to over 1100, with the mean amount being about 55 dollars.

As of 2024, the data has been simulated into 2025, so that it is easier to do demonstrate some of the machine learning capabilities like evaluation in notebooks that you will be using later.

#### Running the query with BigQuery Dataframes
BigQuery DataFrames (Preview) provides a Pythonic DataFrame and machine learning (ML) API powered by the BigQuery engine.

* bigframes.pandas provides a pandas-compatible API for analytics.
* bigframes.ml provides a scikit-learn-like API for ML.
BigQuery DataFrames is an open-source package. You can run pip install --upgrade bigframes to install the latest version

In [61]:
bdf_tx = bpd.read_gbq(f"{PROJECT_ID}.tx.tx")

In [62]:
bdf_tx.head()

,TX_ID,TX_TS,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,43eaf55a264a39b7d68ba96102b231453f0c0682,2023-01-25 02:41:21+00:00,1221481630749342,87156608,14.990000000
1,cb25aada4a7bff390d688f68927fcc04b700aaf6,2023-09-03 09:02:47+00:00,2699340061069102,68885177,26.950000000
2,9c4162197deb562a2a011d45341ae62122a46c5c,2022-04-06 19:12:24+00:00,0985771247432105,03935526,95.300000000
3,57db42f5e59e0055a7d5869e51a27f0b7b5b2bd3,2022-09-26 02:05:59+00:00,7407998643616158,27222215,95.440000000
4,dff274b7ad8c7d3af21b5cf49e830337e82f9391,2023-03-08 12:45:38+00:00,7717343913739160,60482124,80.800000000


In [63]:
bpd.DataFrame({'NUM_TX': [len(bdf_tx)],
              'MIN_TX_DATE': [str(bdf_tx['TX_TS'].min())],
              'MAX_TX_DATE': [str(bdf_tx['TX_TS'].max())],
              'NUM_CUSTOMERS' : [bdf_tx['CUSTOMER_ID'].nunique()],
              'NUM_TERMINALS' : [bdf_tx['TERMINAL_ID'].nunique()],
              'MIN_TX_AMOUNT' : [float(bdf_tx['TX_AMOUNT'].min())],
              'AVG_TX_AMOUNT' : [float(bdf_tx['TX_AMOUNT'].mean())],
              'MAX_TX_AMOUNT' : [float(bdf_tx['TX_AMOUNT'].max())]
             })

,NUM_TX,MIN_TX_DATE,MAX_TX_DATE,NUM_CUSTOMERS,NUM_TERMINALS,MIN_TX_AMOUNT,AVG_TX_AMOUNT,MAX_TX_AMOUNT
0,73968721,2022-01-01 00:00:00+00:00,2024-01-01 00:12:40+00:00,49984,5000,0.0,55.152797,1101.55


#### Get fraud classification counts and percentages
Next, you look at the fraud label data to see how many (and what percentage of) transactions are classified as fraud or not.

In [65]:
run_bq_query(
    """
    SELECT
      TX_FRAUD,

      COUNT(*) AS NUM_TX,

      /* Calculates number of total transactions (not grouped) */
      SUM(COUNT(*)) OVER () AS OVR_TOTAL_TX,

      /* Calculates number of transaction in this group as % of total */
      SAFE_DIVIDE(
        COUNT(*),
        SUM(COUNT(*)) OVER ()
        ) AS PCT_TOTAL_TX

    FROM
      tx.txlabels

    GROUP BY
      TX_FRAUD

    ORDER BY
      TX_FRAUD
    """
)

Finished job_id: d747e219-b184-4989-a47a-05ce382738a6


,TX_FRAUD,NUM_TX,OVR_TOTAL_TX,PCT_TOTAL_TX
0,0,72489095,73968721,0.979997
1,1,1479626,73968721,0.020003


You can see that about 2% of transactions are fraudulent, while the remaining 98% of transactions are not labeled as fraud.

#### Running the query with BigQuery Dataframes

In [89]:
bdf_tx_labels = bpd.read_gbq(f"{PROJECT_ID}.tx.txlabels")
grouped = bdf_tx_labels.groupby('TX_FRAUD')['TX_FRAUD'].count()
total_tx = grouped.sum()
result = grouped.to_frame(name='NUM_TX')
result['OVR_TOTAL_TX'] = total_tx
result['PCT_TOTAL_TX'] = result['NUM_TX'] / result['OVR_TOTAL_TX']

In [91]:
result

,NUM_TX,OVR_TOTAL_TX,PCT_TOTAL_TX
TX_FRAUD,,,
0,72489095,73968721,0.979997
1,1479626,73968721,0.020003


#### Plot transaction amount distribution
Next, you examine the distribution of transaction amounts more closely. After seeing above that the range is about 0 to 1100 dollars, you want to look at how often various transaction amounts across that range show up. Rather than bringing in all ~148M transactions amounts and plotting them, you can aggregate to the nearest dollar and count the number of observations in each dollar "bucket" using BigQuery.

In [92]:
transaction_amount_dist = run_bq_query(
    """
    SELECT
      /* Round transaction to nearest dollar, use to group */
      ROUND(TX_AMOUNT, 0) AS ROUNDED_TX_AMOUNT,
      COUNT(*) AS NUM_TX

    FROM
      tx.tx

    GROUP BY
      ROUNDED_TX_AMOUNT

    ORDER BY
      ROUNDED_TX_AMOUNT
    """
)

transaction_amount_dist

Finished job_id: 5a85abc7-34a3-41db-a400-fa20d6943075


,ROUNDED_TX_AMOUNT,NUM_TX
0,0E-9,12464
1,1.000000000,49388
2,2.000000000,104290
3,3.000000000,190959
4,4.000000000,307669
...,...,...
909,950.000000000,1
910,951.000000000,1
911,960.000000000,1
912,993.000000000,1


You can display the results as a histogram using the functionality to build interactive bar charts in [Plotly Express](https://plotly.com/python/bar-charts/).

In [93]:
# Create histogram of transaction amounts
transaction_amount_interactive_histogram = px.bar(
    transaction_amount_dist,
    x="ROUNDED_TX_AMOUNT",
    y="NUM_TX",
    labels={
        "ROUNDED_TX_AMOUNT": "Transaction Amount (to nearest $)",
        "NUM_TX": "# of Transactions",
    },
    title="Number of Transactions by Amount",
)

transaction_amount_interactive_histogram.show()

You can see that the vast majority of transaction amounts are 125 dollars or less, with a steady "plateau" of 700K-800K transactions at each dollar amount from about 10 to 80. One advantage of the interactive bar chart is that you can zoom in to different areas (e.g. the bars are hard to see past 100 or so on the default plot, so reducing the y-axis helps) and then hover over different bars to see the actual numbers.

#### Analyze customer-level aggregates of transaction data
After looking at the data on individual transactions in a couple different ways, you now look at the transaction data aggregated to the customer level. The query below gets three summary values for each customer in the dataset:
*   number of transactions
*   average transaction value
*   percentage of transactions that are fraudulent

In [94]:
customer_aggregates = run_bq_query(
    """
    SELECT
      CUSTOMER_ID,
      COUNT(*) AS NUM_TX,
      AVG(TX_AMOUNT) AS AVG_TX_AMOUNT,

      /* Get % of transactions that are fraudulent, accounting for
         possibility of missing fraud labels */
      SAFE_DIVIDE(
        SUM(IF(TX_FRAUD IS NOT NULL, TX_FRAUD, 0)),
        SUM(IF(TX_FRAUD IS NOT NULL, 1, 0))
        ) AS PCT_TX_FRAUD

    FROM
      `tx.tx`

    /* Join to labels data to get fraud status of each transaction */
    LEFT JOIN
      `tx.txlabels` USING (TX_ID)

    GROUP BY
      CUSTOMER_ID
    """
)

customer_aggregates.head()

Finished job_id: 63a77bee-1c7f-4d0a-87e5-5791976fc4cf


,CUSTOMER_ID,NUM_TX,AVG_TX_AMOUNT,PCT_TX_FRAUD
0,3421182254942355,2801,10.103181007,0.000357
1,5755162810173312,2142,70.659080299,0.026144
2,4371069907865457,2291,23.109580969,0.022261
3,2333812774760192,2061,6.790203785,0.000485
4,4732264206945358,2325,11.446068817,0.030538


You can now explore this customer data further using [Plotly's scatter plot functionality](https://plotly.com/python/line-and-scatter/) to look at each customer's average transaction value vs number of transactions in an interactive plot. The code below does that, while also coloring points by the percentage of a customer's transactions that are fraudulent.

In [96]:
customer_aggregates_scatter_plot = px.scatter(
    customer_aggregates,
    x="NUM_TX",
    y="AVG_TX_AMOUNT",
    color="PCT_TX_FRAUD",
    labels={
        "CUSTOMER_ID": "Customer ID",
        "NUM_TX": "# of Transactions",
        "AVG_TX_AMOUNT": "Avg Transaction Amount ($)",
        "PCT_TX_FRAUD": "% of Transactions That Are Fraud",
    },
    hover_data=["CUSTOMER_ID"],
    title=(
        "<b>Customer-Level Transaction Aggregates</b><br>"
        + "(Each Point is 1 Customer, Colored by % of Fraud Transactions)"
    ),
)

customer_aggregates_scatter_plot.show()

The plot above shows that the vast majority of customers have less than 3000 transactions with an average value of 100 or less, and most of those appear to have very low fraud rates. But there's a cluster of customers at the far right with a very large number of transactions (in the 3000-6000 range), some of whom who have very high average transaction values (from 100 to more than 300 dollars). And the customers in this group with very high transaction counts also have an extremely high rate of fraud: all upwards of 60%, with some at nearly 100% fraud.

These trends found in the data are definitely worth keeping in mind as you think about how to construct a model to predict fraudulent transactions.

Note that because this is simulated data, the graph is unlikely to be very realistic (!) so consider this more of an exercise in understanding the workflow you may encounter and the products on Google Cloud that can help you solve for real-time fraud classification along the way.

#### Perform clusterization of fraudulents transactions with k-Means

In [119]:
from bigframes.ml.cluster import KMeans
n_clusters = 3
cluster_model = KMeans(n_clusters=n_clusters) 

In [106]:
cluster_model.fit(bpd.DataFrame(customer_aggregates[['NUM_TX','AVG_TX_AMOUNT','PCT_TX_FRAUD']]))

/opt/conda/lib/python3.10/site-packages/ibis/backends/bigquery/__init__.py:499: FutureWarning:

`database` is deprecated as of v7.1, removed in v8.0; The bigquery backend cannot return a table expression using only a `database` specifier. Include a `schema` argument.



KMeans(n_clusters=3)

In [109]:
clustered_result = cluster_model.predict(bpd.DataFrame(customer_aggregates[['NUM_TX','AVG_TX_AMOUNT','PCT_TX_FRAUD']]))
clustered_result.head(n=5)

/opt/conda/lib/python3.10/site-packages/ibis/backends/bigquery/__init__.py:499: FutureWarning:

`database` is deprecated as of v7.1, removed in v8.0; The bigquery backend cannot return a table expression using only a `database` specifier. Include a `schema` argument.



,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,NUM_TX,AVG_TX_AMOUNT,PCT_TX_FRAUD
0,1,"[{'CENTROID_ID': 1, 'DISTANCE': 1.135931355828...",2801,10.103181007,0.000357
1,3,"[{'CENTROID_ID': 3, 'DISTANCE': 0.471969087215...",2142,70.659080299,0.026144
2,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.489710463705...",2291,23.109580969,0.022261
3,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.824810107907...",2061,6.790203785,0.000485
4,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.838748474567...",2325,11.446068817,0.030538


In [110]:
cluster_1_result = clustered_result[clustered_result["CENTROID_ID"] == 1][['NUM_TX','AVG_TX_AMOUNT','PCT_TX_FRAUD']]
cluster_1_result_pandas = cluster_1_result.head(5).to_pandas()

cluster_2_result = clustered_result[clustered_result["CENTROID_ID"] == 2][['NUM_TX','AVG_TX_AMOUNT','PCT_TX_FRAUD']]
cluster_2_result_pandas = cluster_2_result.head(5).to_pandas()

cluster_3_result = clustered_result[clustered_result["CENTROID_ID"] == 3][['NUM_TX','AVG_TX_AMOUNT','PCT_TX_FRAUD']]
cluster_3_result_pandas = cluster_3_result.head(5).to_pandas()

In [135]:
prompts_list = []
prompt1 = 'transaction profile 1:\n'
for i in range(5):
    prompt1 += str(i + 1) + '. ' + \
        cluster_1_result_pandas[['NUM_TX','AVG_TX_AMOUNT','PCT_TX_FRAUD']].iloc[i].to_string() + '\n'
prompts_list.append(prompt1)

prompt2 = 'transaction profile 2:\n'
for i in range(5):
    prompt2 += str(i + 1) + '. ' + \
       cluster_2_result_pandas[['NUM_TX','AVG_TX_AMOUNT','PCT_TX_FRAUD']].iloc[i].to_string() + '\n'
prompts_list.append(prompt2)
    
    
prompt3 ='transaction profile 3:\n'
for i in range(5):
    prompt3 += str(i + 1) + '. ' + \
        cluster_3_result_pandas[['NUM_TX','AVG_TX_AMOUNT','PCT_TX_FRAUD']].iloc[i].to_string() + '\n'
prompts_list.append(prompt3)

In [136]:
 prompt = f"""
    Next, there is the description of {n_clusters} transactions clusters that have been tagged as fraudulents,
    Write a short explanation of the semantics of each type:
    Return in JSON with the following format  "tx_cluster_id": "1" , "tx_profile" : "description", ...,     
    """
for i in range(n_clusters):
    prompt += prompts_list[i]

In [138]:
from bigframes.ml.llm import PaLM2TextGenerator

llm_model = PaLM2TextGenerator(model_name="text-bison-32k")
bdf = bpd.DataFrame({"prompt": [prompt]})
transaction_desc = llm_model.predict(bdf)
print(transaction_desc["ml_generate_text_llm_result"].iloc[0])

 ```json
{
  "tx_cluster_id": "1",
  "tx_profile": "This cluster is characterized by a high number of transactions with relatively low average amounts and a low percentage of fraudulent transactions."
},
{
  "tx_cluster_id": "2",
  "tx_profile": "This cluster is characterized by a moderate number of transactions with relatively high average amounts and a moderate percentage of fraudulent transactions."
},
{
  "tx_cluster_id": "3",
  "tx_profile": "This cluster is characterized by a high number of transactions


### END

Now you can go to the next notebook `02_feature_engineering_batch.ipynb`